# Very brief introduction to the python libraries: `xarray` and `dask`

- `xarray` is similar to the `pandas` library, but for multi-dimensional data. Whereas `pandas` works well with 2-d or tabular data, it is common in oceanography (at least in physical oceanography) to have 3-d or 4-d data (with 3 spatial dimensions and 1 time dimension). `xarray` is also very useful for analyzing large volumes of data often in netcdf (.nc) format.
- `dask` is a library that parallelizes code (i.e. can run multiple computations at the same time) in a relatively easy and efficient way. It is very useful when using big data and integrates nicely with `xarray`.

This notebook is meant to only be a *very brief* introduction to these two libraries, so you know where to start if you'd like to use these tools in your own research/coding. There are a few very nice tutorials online about both of these libraries, including these two by Ryan Abernathey:

- "Xarray Fundamentals": https://rabernat.github.io/research_computing_2018/xarray.html 
- "Dask for Parallel Computing and Big Data": https://rabernat.github.io/research_computing_2018/dask-for-parallel-computing-and-big-data.html#

# Load a dataset

This is a dataset from 41 years of the CESM (Community Earth System Model) ocean model ("POP": Parallel Ocean Program). This dataset is stored on the Cloud through the Pangeo Project, and we can load it directly into this notebook that is running on the Cloud too! 

When we say "load" here, we actually don't load the entire dataset into memory. Instead, we do what is called "lazy loading", where we load only the dataset metadata initially. In this way, we can examine the dataset (which variables, size of variables, names of dimensions, etc.) without having to worry about how large the dataset is. This is very handy when using large datasets (such as the CESM dataset we will use here which is a whopping 5.6  terabytes!), since the dataset is too large to fit into any computer's memory. 

But, often we are only interested in one variable (say, sea surface temperature) or a specific spatial region or time frame, and so we only need a small subset of the larger dataset. `xarray` and `dask` allow us to work with large datasets and only load in the portion of the data that we actually are interested in, making it possible to do data analysis on very large datasets.

We load in the (meta)data below using the library `intake`, which (among other features) provides a quick and standardized way to load data from a catalog. You can read more about `intake` [here](https://intake.readthedocs.io/en/latest/).

In [ ]:
from intake import open_catalog

cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean/CESM_POP.yaml")
ds  = cat["CESM_POP_hires_control"].to_dask()

# xarray

#### To view details of the dataset, simply type its name (we called it `ds` above)

In [ ]:
ds

#### So you can see that the data has a few things to note:

- the data is a `xarray.Dataset` type
- 8 dimensions of different sizes: nlat, nlon, time, ...
- 59 coordinates: time, along with many others
- 12 variables: SST, and many others
- a list of attributes (i.e. metadata about the dataset: full name of model run, time format, etc.)

The above output is actually interactive, so you can click on the arrows to the lft of the words (e.g. the arrow to the left of "Data variables" to view a list of variables stored in this dataset. You can also click on the icons on the right end of each line. So, if you want to know what the variable SST is, click the paper icon on the right and it will tell you that it is the "Surface Potential Temperature" in degrees Celsius ("degC").

The icon that looks like stacked cylinders immediately to the right of the paper icon is also clickable, and shows information about the size, shape, and type of data in that variable. Notice that it has two columns: one for the full variable array and one for single chunks of the DataArray. As you can see, the chunks are much smaller in size than the full array, and chunking up arrays into small, manageable sizes is one of the main ways we can reduce computations on large datasets into doable actions.

You can start to understand just how useful xarray, as it can load and display all different types of information about a dataset.

To access items within the dataset, we just type `ds.` followed by the aspect of the dataset we are interested in. The following few cells show a few examples.

See information about the variable 'SST':

In [ ]:
ds.SST

Notice that SST is an `xarray.DataArray`, as opposed to a `xarray.Dataset` like our `ds` above. In `xarray`, a Dataset is a collection of DataArrays. So typically, each variable and coordinate is a DataArray, and all together they make up a Dataset.

List information about the time dimension. This model uses `cftime` - a python library for denoting time - and uses a calendar without leap years. Hence the `cftime.DateTimeNoLeap` objects:

In [ ]:
ds.time

You can see that the first three timesteps start with the number 46. In this model, 46 is the starting model year, while the numbers following are the month and day, respectively. So this dataset starts on January 2nd, in the 46th year of the model run, and ends on January 1st, year 87 of the model. So this tells us that we have ~41 years of data, at daily output.

List all dimensions:

In [ ]:
ds.dims

There are quite a few dimensions in a global ocean model like this. Here, `nlat` and `nlon` are the number of points in the latitudinal and longitudinal directions (i.e. not the lat/lon values themselves).

One of the great features of xarray is that it is aware of the names of dimensions. So if, for instance, you'd like to grab the output from only the first day of the dataset, you can do so as follows:

In [ ]:
# Grab the SST output from the first day only 
ds.SST.isel(time=0)

Notice how we don't need to specify which axis number `time` is - we need only write 'time' and xarray knows which dimension that is. The `.isel()` functions selects the specified range (in the above case, the 0th index of `time`). We can also grab a larger range of values, e.g. by grabbing the first 10 days of SST:

In [ ]:
# Grab the SST output from the first day only 
ds.SST.isel(time=slice(0,10))

Xarray also allows for some simple data manipulation, such as taking the mean of a dataset. First we select the SST variable, and then we write out the function `.mean()` with the argument 'time' to take the time average.

In [ ]:
ds.SST.mean('time')

Great, so we just took the mean of a dataset in one line! What if we want to plot some data? Let's try to plot the first day of SST data. All you have to do is add `.plot()` to the end of the line!

In [ ]:
# Note: this may take several seconds
ds.SST.isel(time=0).plot()

You may have noticed that this sook several seconds. Until now, we hadn't actually *loaded* any of the data. We had just loaded metadata - enough for us to look at the data and understand it a bit better. But, to make a plot, we need to actually access the data, and so adding `.plot()` triggers a load of the data, but only what is needed to make the plot. Since we only plotted the first day, it was reasonably quick. 

Side note: This plot can look much spiffier with the python library `cartopy`! If you want to try out `cartopy`, go through to the cartopy tutorial that Josué made for you and attend his tutorial!

#### Ok, that is all we will do with xarray for now. I hope you can see how useful it can be, and you will be making use of xarray in the ECCO tutorials!

You can also check out the `xarray` tutorial linked at the top of this notebook - it goes into much more detail!

# Dask

## In this brief dask tutorial (largely based on [this one](https://tutorial.dask.org/00_overview.html)), we will see how dask can help speed up your computations.

### Dask has two parts:
1. distributed computing for running code in parallel
2. helping with large-than-memory datasets and lazy loading/computations - this works with libraries like `xarray`, as well as `pandas`

**1. Parallel computing**

We will start by defining some very basic adding and multiplying functions that use the function `sleep()` from the `time` library. This sleep() function causes the function add() that we define below to pause for the number of seconds inside the parentheses."

In [ ]:
from time import sleep

# Define a function to add two numbers
def add(x,y):
    sleep(2) # pause for 2 seconds
    return x+y

# Define a function to multiply two numbers
def mlt(x,y):
    sleep(1) # pause for 1 second
    return x*y

In the next cell, we will use another 'magic' function similar to the `%matplotlib inline` function you have probably used numerous times. These function calls that begin with a `%` are called 'magic' functions in ipython notebooks. This time we will use `%%time`, which prints out the amount of time it takes to run all of the code in that particular cell.

If we call add() once and mlt() twice, can you guess how long it will take to run? It should take almost exactly 2 + 1 + 1 seconds.

In [ ]:
%%time

a = add(1,2)
b = mlt(1,2)
c = mlt(a,b)

On my JupyterHub instance, it took 4.0 seconds to run - exactly as expected! 

But, we could theoretically run all of these functions at the same time, since they are all independent calculations. `dask` can help us do that! We are now going to import a dask function called 'delayed'. It is so called because it doesn't run a function immediately, but stores the information to run a function until the user specifies a `.compute()` function, at which point the calculation is run, and in the most optimized way. Let's see an example:

In [ ]:
from dask import delayed

In [ ]:
%%time

a = delayed(add)(1,2)
b = delayed(mlt)(1,2)
c = delayed(mlt)(a,b)

Hmm, this claims that the calculations ran in 358 microseconds - but that doesn't make sense! Our two functions require at least a 1 second pause when run. The catch is that we haven't actually done the calculation yet. We have just created delayed objects that will run once we compute them.

In [ ]:
%%time

c.compute()

Now it looks like it only took 3.07 seconds to run the same calculation as before when it took 4 seconds! While 1 second doesn't sound like much, this can be scaled up with large amounts of data. You can save hours or even days or your time by parallelizing with dask!

**2. Dask with big data**

Now we will briefly look at the second aspect of Dask. Take a look at our CESM dataset that we loaded during the xarray portion of this tutorial. Let's view it again here:

In [ ]:
ds

Click on the arrow to the left of the "Data variables", and you should see the list of variables appear. If you read each variable line, you will see that it says `dask.array<chunksize=(1, 2400, 3600)`. What this tells us is that each `xarray.DataArray` actually wraps a `dask.array`. So xarray and dask are intertwined, and work together to make data analysis of large datasets possible. 

Check out the ECCO notebooks and tutorials for more practice using `xarray` and `dask`, with some real examples of analyzing a global ocean dataset!

If you want to learn more about Dask, go through the tutorial linked at the top of the page.

Note that dask may not be necessary to use if you do not have big data! If you do have large amounts of data that take a long time to run, then dask is a great resource, and you can use it on your local computer, on high performance computing infrastructures, or on the cloud!